In [ ]:
#!/usr/bin/env python3.9 -m nuitka
# -*- coding: utf-8 -*-
import re
import os
import sys
import json
import time
import urllib3
import platform
import requests
import unicodedata
import configparser
import browser_cookie3
from pprint import pprint
from datetime import datetime
from playsound import playsound
from sdk.api.message import Message
from sdk.exceptions import CoolsmsException

urllib3.disable_warnings()

def get_header(cate):
    if cate=="map":
        headers = {
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/json;charset=utf-8",
            "Origin": "https://vaccine-map.kakao.com",
            "Accept-Language": "en-us",
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.6",
            "Referer": "https://vaccine-map.kakao.com/",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "Keep-Alive",
            "Keep-Alive": "timeout=5, max=1000",
            "Cookie" : cookie_from_kakaotalk
        }
    else:
        headers = {
            "Accept": "application/json, text/plain, */*",
            "Content-Type": "application/json;charset=utf-8",
            "Origin": "https://vaccine.kakao.com",
            "Accept-Language": "en-us",
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.5",
            "Referer": "https://vaccine.kakao.com/",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "Keep-Alive",
            "Keep-Alive": "timeout=5, max=1000",
            "Cookie" : cookie_from_kakaotalk
        }
    return headers

def check_user_info_loaded():
    user_info_api = 'https://vaccine.kakao.com/api/v1/user'
    user_info_response = requests.get(
        user_info_api, headers=get_header("vac"), verify=False)
    user_info_json = json.loads(user_info_response.text)
    if user_info_json.get('error'):
        print("사용자 정보를 불러오는데 실패하였습니다. 카카오톡 쿠키 값을 확인해 보세요.")
        print("쿠키값이 정상이라면, 카카오톡에 들어가서 잔여백신 알림 신청을 한번 해보세요. 정보제공 동의가 나온다면 동의 후 다시 시도해주세요.")
        close(None)
    else:
        user_info = user_info_json.get("user")
        for key in user_info:
            value = user_info[key]
            # print(key, value)
            if key != 'status':
                continue
            if key == 'status' and value == "NORMAL":
                print("{} : 사용자 정보를 불러오는데 성공했습니다. (상태 : {})".format(user_info_json["user"]["name"], user_info_json["user"]["status"]))
                break
            elif key == 'status' and value == "UNKNOWN":
                print("상태를 알 수 없는 사용자입니다. 1339 또는 보건소에 문의해주세요.")
                close(None)
            else:
                print("이미 접종이 완료되었거나 예약이 완료된 사용자입니다.")
                close(None)

def close(success=False):
    if success is True:
        playsound("tada.mp3")
        print("잔여백신 예약 성공!! 카카오톡지갑을 확인하세요.")
        send_lms("잔여백신 예약 성공!! 카카오톡지갑을 확인하세요.")
    elif success is False:
        print("잔여백신 발견 성공, 하지만 예약 중 오류와 함께 종료되었습니다.")
        send_lms("잔여백신 발견 성공, 하지만 예약 중 오류와 함께 종료되었습니다.")
    else:
        pass
    input("Press Enter to close...")
    sys.exit()

def send_lms(message):
    global hp_number, api_secret
    # set api key, api secret
    api_key = "NCS57256296AFC9B"

    ## 4 params(to, from, type, text) are mandatory. must be filled
    params = dict()
    params["type"] = "sms" # Message type ( sms, lms, mms, ata )
    params["to"] = hp_number # Recipients Number "01000000000,01000000001"
    params["from"] = hp_number # Sender number
    params["text"] = message # Message

    cool = Message(api_key, api_secret)
    try:
        response = cool.send(params)
        print("Success Count : %s" % response["success_count"])
#         print("Error Count : %s" % response["error_count"])
#         print("Group ID : %s" % response["group_id"])
        if "error_list" in response:
            print("Error List : %s" % response["error_list"])
    except CoolsmsException as e:
        print("Error Code : %s" % e.code)
        print("Error Message : %s" % e.msg)

def try_reservation(organization_code, vaccine_type):
    reservation_url = 'https://vaccine.kakao.com/api/v2/reservation'
    data = {"from": "Map", "vaccineCode": vaccine_type,
            "orgCode": organization_code, "distance": None}
    response = requests.post(reservation_url, data=json.dumps(
        data), headers=get_header("vac"), verify=False)

    f = open("response.txt", "w") # 파일 열기
    print(response.text, file=f) # 파일 저장하기
    f.close()

    response_json = json.loads(response.text)
    for key in response_json:
        value = response_json[key]
        if key != 'code':
            continue
        if key == 'code' and value == "NO_VACANCY":
            print("접종 신청이 선착순 마감되어 잔여백신 신청에 실패했습니다.")
            close(None)
        elif key == 'code' and value == "TIMEOUT":
            print("TIMEOUT, 잔여백신 신청에 실패했습니다.")
            close(None)
        elif key == 'code' and value == "SUCCESS":
            print("백신접종신청에 성공했습니다.")
            organization_code_success = response_json.get("organization")
            print("병원이름 : {}\n전화번호 : {}\n종료시간 : {}\n예약백신 : {}\n주소 : {}\n".format(
                organization_code_success.get('orgName'), 
                organization_code_success.get('phoneNumber'), 
                response_json["organization"]["openHour"]["openHour"]["end"], 
                response_json.get('vaccineName'), 
                organization_code_success.get('address'))
                 )
            close(success=True)
        else:
            print("ERROR. 아래 메시지를 보고, 예약이 신청된 병원 또는 1339에 예약이 되었는지 확인해보세요.")
            print(response.text)
            close(None)

def find_vaccine():
    global search_time, reserve_run, except_list, vaccine_want, min_end_time, end_type
    global top_x, top_y, bottom_x, bottom_y, only_left
    try_count = 0
    done = False

    url = 'https://vaccine-map.kakao.com/api/v3/vaccine/left_count_by_coords'
    data = {"bottomRight": {"x": bottom_x, "y": bottom_y}, "onlyLeft": only_left, "order": "latitude",
            "topLeft": {"x": top_x, "y": top_y}}
    while True:
        time.sleep(search_time)
        if done is True:
            break
        else:
            try_count += 1
            not_want_hosp = []
            try:
                response = requests.post(url, data=json.dumps(
                    data), headers=get_header("map"), verify=False, timeout=5)

                try:
                    json_data = json.loads(response.text)
#                     pprint(json_data)
                    print("{}번째 시도중.............".format(try_count), end="\r")

                    for x in json_data["organizations"]:
                        if done is True:
                            break
                        if x.get('orgName') in except_list:
                            pass
                        elif x.get('status') == "AVAILABLE" or x.get('leftCounts') != 0:
                            organization_code = x.get('orgCode')
                            check_organization_url = f'https://vaccine.kakao.com/api/v3/org/org_code/{organization_code}'
                            check_organization_response = requests.get(check_organization_url, headers=get_header("map"), verify=False)
                            hosp = json.loads(check_organization_response.text)
                            end_time = int(hosp["organization"]["openHour"]["openHour"]["end"].split(":")[0])

                            # 실제 백신 남은수량 확인
                            vaccine_found_code = None
                            for vac_info in hosp.get("lefts"):
                                if vac_info.get('leftCount') != 0:
                                    if vac_info.get('vaccineName') in vaccine_want:
                                        if end_time >= min_end_time:
                                            if reserve_run:
                                                print("{}번째 시도중... {} : {} 백신을 {}개 발견했습니다.".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')))
                                                print("주소 : {} 입니다.".format(hosp["organization"]["address"]))
                                                vaccine_found_code = vac_info.get('vaccineCode')
                                                done = True
                                                playsound("wow.wav")
                                                try_reservation(organization_code, vaccine_found_code)
                                                break
                                            else:
                                                print("{}번째 시도중... {} : {} 백신을 {}개 발견하였으나 예약은 진행하지 않습니다.".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')))
                                        else:
                                            print("{}번째 시도중... {} : {} 백신을 {}개 발견하였으나 시간이 맞지 않아 예약은 진행하지 않습니다.".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')))
                                    else:
                                        not_want_hosp.append(hosp["organization"]["orgName"])
                                        print("{}번째 시도중... 원하는 백신이 아니어서 패스합니다 : {}({} {}개)".format(try_count, hosp["organization"]["orgName"], vac_info.get('vaccineName'), vac_info.get('leftCount')), end=end_type)

                except json.decoder.JSONDecodeError as decodeerror:
                    print("JSONDecodeError : ", decodeerror)
                    print("JSON string : ", response.text)
                    close()
            except requests.exceptions.Timeout as timeouterror:
                print("Timeout Error : ", timeouterror)
            except requests.exceptions.SSLError as sslerror:
                print("SSL Error : ", sslerror)
                close()
            except requests.exceptions.ConnectionError as connectionerror:
                print("Connection Error : ", connectionerror)
                # See psf/requests#5430 to know why this is necessary.
                if not re.search('Read timed out', str(connectionerror), re.IGNORECASE):
                    close()
            except requests.exceptions.HTTPError as httperror:
                print("Http Error : ", httperror)
                close()
            except requests.exceptions.RequestException as error:
                print("AnyException : ", error)
                close()
            
            if len(not_want_hosp) > 0:
                f = open("not_want_hosp.txt", "w") # 파일 열기
                print(",".join(not_want_hosp), file=f) # 파일 저장하기
                f.close()

def main_function():
    check_user_info_loaded()
    find_vaccine()
    close()

# ===================================== run ===================================== #
search_time = 0.2  # 잔여백신을 해당 시간마다 한번씩 검색합니다. 단위: 초
vaccine_want = ["화이자", "모더나"] #원하는 백신 이름을 한글로 배열로 입력 (아스트라제네카, 화이자, 모더나, 얀센)
min_end_time = 0 #18로 할 경우 18시 포함 더 늦은 종료시간인 병원에 해당할 경우에만 예약함
hp_number = "" #문자발송을 위한 정보
api_secret = "" #문자발송을 위한 정보
only_left = "true" #request 시 잔여백신이 있는 병원 정보만 가져오는 파라미터
cookie_from_kakaotalk = "_kavacto=" #카카오톡 웹뷰의 https 접속 이력을 파악해야 가능함

#포항지역 좌표정보
top_x = "129.4369078955544"
top_y = "35.931968680267204"
bottom_x = "129.28111700916696"
bottom_y = "36.059338527137314"

#다른 좌표
# top_x = "127.07440408758595"
# top_y = "37.42121167268209"
# bottom_x = "127.14502465645803"
# bottom_y = "37.37230821920313"

except_list = []
# except_list = open("not_want_hosp.txt", mode="rt", encoding="utf-8").read().replace("\n","").split(",")
end_type = "\r"

reserve_run = True #False로 할 경우 실제 예약은 하지 않고 현황만 보여줌. True로 하면 예약까지 실행
if __name__ == '__main__':
    main_function()